<a href="https://colab.research.google.com/github/Ash4dev/expert-octo-garbanzo/blob/main/side_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
import shutil

for dirpath, dirnames, filenames in os.walk("/content/drive/MyDrive/prel_dataset"):
  for file in filenames:
    shutil.copy(os.path.join("/content/drive/MyDrive/prel_dataset", file),"/content/prel_dataset")

In [ ]:
# it will be easier to learn from the data presented all at once, rather than sequentially 1-2 min
# delimiter_length can even be lesser.

def unify_dataset(data_path, target_path = "/content/unified_dataset", delimiter = '/', delimiter_length = 4):
  """
  purpose: unify the sequential datas (for improved pattern capture of the data by the network)
  params: path where the datafiles are located, path where the unified file is to be stored, *
  returns: unified_string/None decide
  """
  new_score_delimiter = delimiter * delimiter_length   # score_end indicator
  unified_string = ""

  for dirpath, dirnames, filenames in os.walk(data_path):
    for index, file in enumerate(filenames):
      with open(os.path.join(data_path, str(file)), 'r') as file_ptr: # the individual files are read
        unified_string += file_ptr.read()
        if index < len(filenames) - 1: # delimiter not to be applied after the last score
          unified_string += " " + new_score_delimiter + ' '


  with open(target_path, 'w') as file_ptr: # saving the unified string to a file
    file_ptr.write(unified_string)
  
  return unified_string


In [ ]:
dataset = []
dataset = unify_dataset("/content/prel_dataset") # set to 32 as exceeding ram otherwise
dataset

'67 _ 64 _ 67 _ 72 _ _ _ _ _ 67 _ 71 _ _ 69 69 _ _ _ _ _ 69 _ 69 _ 72 _ 71 _ _ _ _ _ 65 _ 69 _ _ 67 67 _ _ _ _ _ 67 _ 64 _ 67 _ 76 _ _ _ _ _ 76 _ 74 _ _ 69 74 _ _ _ _ _ 72 _ 71 _ _ 69 67 _ 67 _ 69 _ 71 _ 72 _ _ _ _ _ _ _ r _ 67 _ 64 _ 67 _ 76 _ _ _ _ _ 76 _ 74 _ _ 69 74 _ _ _ _ _ 72 _ 71 _ _ 69 67 _ 67 _ 69 _ 71 _ 72 _ _ _ _ _ _ _ r _ //// 60 _ _ _ 64 _ _ _ 67 _ 64 _ 62 _ _ _ 59 _ _ _ 55 _ _ _ 60 _ _ _ 64 _ 60 _ 67 _ 64 _ 62 _ _ _ _ _ _ _ r _ _ _ 65 _ _ _ 65 _ _ _ 65 _ _ _ 64 _ _ _ _ _ 62 _ 60 _ _ _ 62 _ _ _ 60 _ _ _ 59 _ _ _ 60 _ 62 _ 64 _ 65 _ 67 _ _ _ 69 _ _ _ 69 _ _ _ 69 _ _ _ 67 _ _ _ _ _ 65 _ 64 _ _ _ 67 _ 65 _ 64 _ _ _ 62 _ _ _ 60 _ _ _ _ _ _ _ r _ _ _ //// 60 _ _ _ 67 _ _ _ _ _ _ _ 65 _ 64 _ 62 _ _ _ _ _ _ _ 64 _ _ _ 65 _ _ _ _ _ _ _ 64 _ 62 _ 62 _ _ _ _ _ _ _ _ _ _ _ 62 _ _ _ 62 _ _ _ 60 _ _ _ 59 _ _ _ 62 _ _ _ _ _ _ _ 59 _ _ _ 57 _ _ _ 59 _ _ _ 55 _ _ _ _ _ _ _ _ _ _ _ 55 _ _ _ 57 _ _ _ 59 _ _ _ 60 _ _ _ _ _ _ _ 65 _ _ _ 64 _ _ _ 62 _ _ _ 64 _ _ _ 60 _ _ _ _ _ _ _ //// 79 79 

In [ ]:
set(dataset.split())

{'////',
 '100',
 '103',
 '40',
 '41',
 '43',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '93',
 '95',
 '96',
 '98',
 '_',
 'r'}

In [ ]:
with open("/content/dataset", 'w') as handle:
  handle.write(dataset)

In [ ]:
from google.colab import files
import json

In [ ]:
# 'r', '/'. ' ' and '_' are not understandable by the NN, need to convert them into appropriate format.

def create_mapping(dataset, mapping_path = "mapping.json"):
  """
  purpose: create NN understable data(only numerical)
  params: unified_string, location where the map is to be stored
  returns: None
  """
  dictionary = list(set(dataset.split(" "))) # identify all unique events 
  print(dictionary)
  
  map = {}
  for index, event in enumerate(dictionary): # creating the mapping
    map[event] = index

  with open(mapping_path, 'w') as file_ptr:
    json.dump(map, file_ptr)

In [ ]:
create_mapping(dataset)

['46', '60', '58', '82', '73', '49', '50', '52', '80', '67', '59', '66', '74', '83', '55', '51', '65', '47', '54', '75', '81', '40', '45', '77', '61', '68', '69', '79', '87', '56', '41', '84', '63', '57', '89', '70', '91', '95', '100', '71', '98', '_', '////', '85', '90', '43', '64', '76', '78', '62', '96', '93', '72', '53', 'r', '48', '88', '103', '86']


In [ ]:
files.download("mapping.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
map = {}
with open("mapping.json", 'r') as file_ptr:
  map = json.load(file_ptr) 

In [ ]:
map

In [ ]:
# mapping our songs to map we generated

def NN_feedable_data_convertor(unified_string):
  """
  purpose: converts string to list(feedable to NN)
  params: unified_string
  returns: NN feedable data
  """
  feedable_data = []
  count = 0
  # unified_list = unified_string.split(" ")
  unified_list = dataset.split(" ")
  for event in unified_list:
    count += 1
    if count % 20000 == 0: print(count)
    feedable_data.append(map[event])

  return feedable_data

In [ ]:
feedable_data = NN_feedable_data_convertor(dataset)

20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
260000
280000
300000
320000
340000
360000
380000
400000
420000
440000
460000
480000
500000
520000
540000
560000
580000
600000
620000
640000
660000
680000
700000
720000
740000
760000
780000
800000
820000
840000
860000
880000
900000
920000
940000
960000
980000
1000000
1020000
1040000
1060000
1080000
1100000
1120000
1140000
1160000
1180000
1200000
1220000
1240000
1260000
1280000
1300000
1320000
1340000
1360000
1380000
1400000
1420000
1440000
1460000
1480000
1500000
1520000


In [ ]:
# https://stackoverflow.com/a/33245595/19817062

import pickle
your_data = feedable_data

# Store data (serialize)
with open('feedable_data_4.pickle', 'wb') as handle:
    pickle.dump(your_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
import numpy as np

In [ ]:
# practice
def one_hot_encode(list_data, vocab_len = len(map)):
  """
  purpose: one-hot encode data
  params: list to be one-hot encoded, 
  returns: encoded data
  """
  final_data = np.array([])

  for event in list_data:
      event_data = np.array([0] * vocab_len)
      event_data[event] = 1
      np.append(final_data, event_data)
      final_data.append(event_data)

  return final_data

In [ ]:
a = np.array([3,4,5,6,21,1,[3,4,5]], dtype = object)
a[1] = 34
a

array([3, 34, 5, 6, 21, 1, list([3, 4, 5])], dtype=object)

if error occurs use feedable_data_4.pickle and mappings_4.json

In [ ]:
# 32 notes at once: 2 bars as input; use this to predict the next note; slide the window forward and predict next; till end  
# https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/

def generate_training_sequences(feedable_data, batch_size = 32): # 64 causing crash
  inputs = []
  targets = []

  for count in range(len(feedable_data) - batch_size):
    inputs.append(feedable_data[count: count + batch_size]) # input features one at a time: inside
    targets.append(feedable_data[count + batch_size]) # correspondingtarget value: inside

  inputs = np.array(inputs) # very important to convert to array; list causes crash; large size
  targets = np.array(targets)

  return inputs, targets

In [ ]:
inputs, targets = generate_training_sequences(feedable_data, 32)

In [ ]:
# https://stackoverflow.com/a/33245595/19817062

import pickle
your_data = inputs

# Store data (serialize)
with open('inputs_4.pickle', 'wb') as handle:
    pickle.dump(your_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import shutil
shutil.copy("/content/inputs_4.pickle","/content/drive/MyDrive/pickles_melody")

'/content/drive/MyDrive/pickles_melody/inputs_4.pickle'

In [ ]:
# https://stackoverflow.com/a/33245595/19817062

import pickle
your_data = targets

# Store data (serialize)
with open('targets_4.pickle', 'wb') as handle:
    pickle.dump(your_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import shutil
shutil.copy("/content/targets_4.pickle","/content/drive/MyDrive/pickles_melody")

'/content/drive/MyDrive/pickles_melody/targets_4.pickle'

In [ ]:
inputs

array([[ 9, 41, 46, ..., 41, 39, 41],
       [41, 46, 41, ..., 39, 41, 41],
       [46, 41,  9, ..., 41, 41, 41],
       ...,
       [41, 52, 41, ..., 41, 41, 41],
       [52, 41, 41, ..., 41, 41, 41],
       [41, 41, 41, ..., 41, 41, 54]])

In [ ]:
targets

array([41, 41, 41, ..., 41, 54, 41])

In [ ]:
len(map)

59

In [ ]:
from tensorflow import keras as keras

In [ ]:
one_hot_encoded_inputs = keras.utils.to_categorical(inputs, num_classes = len(map))

In [ ]:
!unzip /content/drive/MyDrive/scores.zip -d /content/expert-octo-garbanzo

In [ ]:
final = []
for input in inputs:
  final.append(one_hot_encode(input))

KeyboardInterrupt: ignored

In [ ]:
!unzip /content/drive/MyDrive/pickles_melody/b.zip -d /content/expert-octo-garbanzo

Archive:  /content/drive/MyDrive/pickles_melody/b.zip
  inflating: /content/expert-octo-garbanzo/b.npy  


In [ ]:
np.load(file_pat, with_pickle = True)